In [ ]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

n_features = 1

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 11 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
st_dims = 8
ac_dims = 4
rw_dims = 1
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

In [ ]:
cols = ["Unnamed: 0", "X Coordinate", "Y Coordinate", "Linear Velocity X", "Linear Velocity Y", "Angle", \
       "Angular Velocity", "Leg1", "Leg2", "index"]
renames = {'Action':'actions', 'Reward':'rewards', 'observation_x':'observations', 'observation_y':'next_observations'}

def get_data():
    dataframe = pd.read_csv("medium play.csv")
    dataframe['observation'] = list(dataframe.iloc[:, 1:9].values)
    observations = dataframe['observation'].copy()
    observations.drop(observations.index[0], inplace=True)
    dataframe.drop(dataframe.index[-1], inplace=True)
    observations = observations.reset_index()
    merged_data = dataframe.merge(observations, left_index=True, right_index=True)
    merged_data["terminals"] = False
    merged_data.loc[184753:, "Episode"] += 2000
    grouped_episodes = merged_data.groupby("Episode")
    merged_data = merged_data.rename(columns=renames)
    for episode, group in grouped_episodes:
        last_index = group.index[-1]  # Get the index of the last row in the group
        merged_data.loc[last_index, 'terminals'] = True
        every = group.index
        merged_data.loc[every, 'rewardstg'] = merged_data.loc[every, 'rewards'].cumsum()
        merged_data.loc[every, 'rewardstg'] -= merged_data.loc[every[-1], 'rewardstg']
        merged_data.loc[every, 'rewardstg'] *= -1
    merged_data.drop(cols, inplace=True, axis=1)
    observations = np.vstack(merged_data['observations'])
    next_observations = np.vstack(merged_data['next_observations'])
    actions = torch.tensor(merged_data["actions"].values, dtype=torch.long)
    rewards = torch.tensor(merged_data["rewardstg"].values, dtype=torch.float)
    observations = torch.tensor(observations, dtype=torch.float)
    next_observations = torch.tensor(next_observations)
    return merged_data, actions, rewards, observations


data, actions, rewards, observations = get_data()



In [ ]:
print(actions.shape, rewards.shape, observations.shape)

torch.Size([400093]) torch.Size([400093]) torch.Size([400093, 8])


In [ ]:
def get_batch(data, ix):
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

def get_intercalated():
    ix = torch.randint(len(data) - block_size, (batch_size,))
    ax, ay = get_batch(actions, ix)
    rx, ry = get_batch(rewards, ix)
    sx, sy = get_batch(observations, ix)
    return ax, ay, rx, ry, sx, sy

@torch.no_grad()
def estimate_loss():
    out = {}
    dt.eval()
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
        ax, ay, rx, ry, sx, sy = get_intercalated()
        logits, loss = dt(sx, ax, rx, targets=ay)
        losses[k] = loss.item()
    out = losses.mean()
    dt.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(3*block_size, 3*block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class DecisionTransformer(nn.Module):

    def __init__(self):
        super().__init__()
        self.st_embd = nn.Linear(st_dims, n_embd)
        self.ac_embd = nn.Embedding(ac_dims, n_embd)
        self.rw_embd = nn.Linear(rw_dims, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, ac_dims)

    def forward(self, states, actions, rewards, targets=None):
        x = self.embeddings(states, actions, rewards)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)

        acs = x[:, 2::3, :]
        preds = self.lm_head(acs) # (B,T,n_features)

        if targets is None:
            loss = None
        else:
            B, T, C = preds.shape
            preds = preds.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(preds, targets)

        return preds, loss

    def embeddings(self, states, actions, rewards):
        states = self.st_embd(states)
        actions = self.ac_embd(actions)
        rewards = self.rw_embd(rewards.unsqueeze(-1))
        B, T, C = states.shape
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        #RSA
        intercalated = torch.tensor(np.zeros((B, 3*T, C)), dtype=torch.float, device=device)
        intercalated[:, 0::3,:] = rewards + pos_emb
        intercalated[:, 1::3, :] = states + pos_emb
        intercalated[:, 2::3, :] = actions + pos_emb

        return intercalated

In [ ]:
dt = DecisionTransformer().to(device)
ax, ay, rx, ry, sx, sy = get_intercalated()
embedding = dt.embeddings(sx, ax, rx)
print(embedding.shape)
preds, loss = dt.forward(sx, ax, rx, targets=ay)

torch.Size([16, 33, 64])


In [ ]:
optimizer = torch.optim.AdamW(dt.parameters(), lr=learning_rate)

In [ ]:
for _ in range(10):
  for iteration in range(5000):
      # every once in a while evaluate the loss on train and val sets
      if iteration % eval_interval == 0 or iter == max_iters - 1:
          losses = estimate_loss()
          print(f"step {iteration}: train loss {losses:.4f}")

      # sample a batch of data
      ax, ay, rx, ry, sx, sy = get_intercalated()

      # evaluate the loss
      logits, loss = dt(sx, ax, rx, targets=ay)
      optimizer.zero_grad(set_to_none=True)
      loss.backward()
      optimizer.step()

step 0: train loss 0.8205
step 100: train loss 0.8196
step 200: train loss 0.7926
step 300: train loss 0.7870
step 400: train loss 0.7928
step 500: train loss 0.7801
step 600: train loss 0.8071
step 700: train loss 0.7891
step 800: train loss 0.7900
step 900: train loss 0.7846
step 1000: train loss 0.8049
step 1100: train loss 0.7790
step 1200: train loss 0.7872
step 1300: train loss 0.7713
step 1400: train loss 0.8011
step 1500: train loss 0.7864
step 1600: train loss 0.7892
step 1700: train loss 0.7874
step 1800: train loss 0.7510
step 1900: train loss 0.7804
step 2000: train loss 0.7818
step 2100: train loss 0.7733
step 2200: train loss 0.7508
step 2300: train loss 0.7596
step 2400: train loss 0.7719
step 2500: train loss 0.7476
step 2600: train loss 0.7700
step 2700: train loss 0.7648
step 2800: train loss 0.7640
step 2900: train loss 0.7632
step 3000: train loss 0.7673
step 3100: train loss 0.7559
step 3200: train loss 0.8106
step 3300: train loss 0.7699
step 3400: train loss 0.77

KeyboardInterrupt: ignored

In [ ]:
!pip install gymnasium
!pip install swig
!pip install "gymnasium[box2d]"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gymnasium as gym

In [ ]:
# Create policy
dt.eval()

# Create environment
env = gym.make("LunarLander-v2")

for i in range(1):
  # Reset it
  total_reward = 0.0

  #numpy array (C)
  state = env.reset(seed=None)

  # While the episode is not finished
  terminated = False
  time = 0

  desired = -100
  rewards = torch.full((1, rw_dims), desired, dtype=torch.float, device=device)
  states = torch.zeros((1, 1, st_dims), dtype=torch.float, device=device)
  states[0, 0, :] = torch.tensor(state, dtype=torch.float, device=device)
  actions = torch.zeros((1, 1), dtype=torch.long, device=device)
  actions[0, 0] = 0.0

  # define a trigger function, return True to start recording a new video:
  episode_trigger=lambda episode_id: (episode_id%1==0)

  # decorate env with a video recorder:
  env = gym.wrappers.RecordVideo(env, "./video",
          episode_trigger=episode_trigger, video_length=100000)

  env.start_video_recorder()

  while not terminated and time <= 500:
      input_st, input_ac, input_rw = states[:, -block_size:, :], \
              actions[:, -block_size:], rewards[:, -block_size:]

      output, loss = dt(input_st, input_ac, input_rw)
      output = output[:, -1:, :].view(ac_dims)
      probabilities = F.softmax(output[:], dim=0)
      # ---> TODO: how to select an action
      # select action with the greatest probability according to policy
      action = int(torch.argmax(probabilities))
      # One step forward
      state, reward, terminated, _ = env.step(action)

      total_reward += reward
      desired -= reward
      state = torch.tensor(state, dtype=torch.float, device=device).view(1, 1, st_dims)
      states = torch.cat((states, state), dim=1)

      reward = torch.tensor(desired, dtype=torch.float, device=device).view(1, rw_dims)
      rewards = torch.cat((rewards, reward), dim=1)

      action = torch.tensor(action, dtype=torch.long, device=device).view(1, 1)
      actions = torch.cat((actions, action), dim=1)

      time += 1

  # Print reward
  print("total_reward = {}".format(total_reward))
env.close()

  deprecation(

  deprecation(

  logger.warn(

  logger.deprecation(

See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(



total_reward = -602.5747525008724


In [ ]:
context = torch.zeros((1, 1,1), dtype=torch.long, device=device)
context[:, -block_size:]

tensor([[[0]]])

In [ ]:
torch.save(dt, "first_iter.model")

In [ ]:
env = gym.make("CartPole-v1")

# define a trigger function, return True to start recording a new video:
episode_trigger=lambda episode_id: (episode_id%1==0)

# decorate env with a video recorder:
env = gym.wrappers.RecordVideo(env, "./video",
         episode_trigger=episode_trigger, video_length=100000)

env.start_video_recorder()